1. Cài đặt thư viện cần thiết

In [ ]:
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q torch torchaudio tqdm ffmpeg transformers datasets peft accelerate

2. Kết nối Google Drive (nếu dữ liệu ở Drive)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


3. Thay đổi đường dẫn đến thư mục chứa file audio

In [ ]:
DATA_DIR = "/content/drive/MyDrive/Whisper/Data_Goc"  # Thư mục chứa file .wav/.mp3 và .txt (transcripts)
MODEL_DIR = "/content/drive/MyDrive/Whisper/pre-train-model"  # Thư mục lưu mô hình

4. Import thư viện cần thiết

In [ ]:
import os
import torch
import whisper
import torchaudio
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from peft import LoraConfig, get_peft_model
from torch.optim import AdamW

In [ ]:
!pip install --upgrade transformers torch accelerate

5. Kiểm tra thiết bị

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("🔹 Using device:", device)

🔹 Using device: cuda


6. Load mô hình Whisper (có thể thay bằng "base", "small", "medium", "large")

In [ ]:
model_name = "openai/whisper-small"
model = WhisperForConditionalGeneration.from_pretrained(model_name).to(device)
processor = WhisperProcessor.from_pretrained(model_name)

7. Áp dụng LoRA vào mô hình

In [ ]:
lora_config = LoraConfig(
    r=16,  # Rank của LoRA (giá trị nhỏ hơn giúp tiết kiệm tài nguyên)
    lora_alpha=32,  # Hệ số scale
    lora_dropout=0.1,
    bias="none",
    target_modules=["q_proj", "v_proj"],  # Các module cần áp dụng LoRA
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 1,769,472 || all params: 243,504,384 || trainable%: 0.7267


8. Chuẩn bị dữ liệu từ thư mục

In [ ]:
def load_data(data_dir):
    audio_files = []
    transcripts = {}

    for file in os.listdir(data_dir):
        if file.endswith(".wav") or file.endswith(".mp3"):
            audio_files.append(os.path.join(data_dir, file))
        elif file.endswith(".txt"):
            with open(os.path.join(data_dir, file), "r", encoding="utf-8") as f:
                transcripts[file.replace(".txt", "")] = f.read().strip()

    return audio_files, transcripts

audio_files, transcripts = load_data(DATA_DIR)
print(f"🔹 Found {len(audio_files)} audio files.")

🔹 Found 2000 audio files.


 9. Huấn luyện mô hình với LoRA

In [ ]:
optimizer = AdamW(model.parameters(), lr=1e-5)

for epoch in range(10):  # Chạy 10 epochs
    for audio_path in audio_files:
        file_name = os.path.basename(audio_path).split(".")[0]
        if file_name not in transcripts:
            continue  # Bỏ qua file không có transcript

        # Load file audio
        audio = whisper.load_audio(audio_path)
        inputs = processor(audio, return_tensors="pt", sampling_rate=16000)
        input_features = inputs.input_features.to(device)

        # Load transcript tương ứng
        labels = processor.tokenizer(transcripts[file_name], return_tensors="pt",truncation=True, max_length=448).input_ids.to(device)

        # Tính loss và tối ưu
        loss = model(input_features, labels=labels).loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        print(f"🔹 Epoch {epoch+1} - Loss for {file_name}: {loss.item()}")

10. Lưu mô hình fine-tuned

In [ ]:
model.save_pretrained(MODEL_DIR)
processor.save_pretrained(MODEL_DIR)
print(f"✅ Mô hình đã được lưu tại: {MODEL_DIR}")

✅ Mô hình đã được lưu tại: /content/drive/MyDrive/Whisper/pre-train-model


11. lưu lần 2

In [ ]:
processor.save_pretrained("/content/drive/MyDrive/Whisper/Final")
merged_model = model.merge_and_unload()
merged_model.save_pretrained("/content/drive/MyDrive/Whisper/Final")

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


In [ ]:
test_audio = "/content/drive/MyDrive/Whisper/Data_LocNhieu/database_sa1_Jan08_Mar19_cleaned_utt_0000000005-1.wav"
result = adapter_model.transcribe(test_audio)
print("🔹 Kết quả nhận dạng:", result["text"])

NameError: name 'adapter_model' is not defined

In [ ]:
!apt-get install unrar
!unrar x "/content/drive/MyDrive/Whisper/Datasets_1_loc_nhieu.rar" "/content/drive/MyDrive/Whisper/Datasets_1_loc_nhieu"

Streaming output truncated to the last 5000 lines.
Extracting  /content/drive/MyDrive/Whisper/Datasets_1_loc_nhieu/Datasets_1_loc_nhieu/database_sa2_Feb29_Mar06_Mar17_toApril06_cleaned_utt_0000187108-1.wav      85%  OK 
Extracting  /content/drive/MyDrive/Whisper/Datasets_1_loc_nhieu/Datasets_1_loc_nhieu/database_sa2_Feb29_Mar06_Mar17_toApril06_cleaned_utt_0000187120-1.txt      85%  OK 
Extracting  /content/drive/MyDrive/Whisper/Datasets_1_loc_nhieu/Datasets_1_loc_nhieu/database_sa2_Feb29_Mar06_Mar17_toApril06_cleaned_utt_0000187120-1.wav      85%  OK 
Extracting  /content/drive/MyDrive/Whisper/Datasets_1_loc_nhieu/Datasets_1_loc_nhieu/database_sa2_Feb29_Mar06_Mar17_toApril06_cleaned_utt_0000187188-1.txt      85%  OK 
Extracting  /content/drive/MyDrive/Whisper/Datasets_1_loc_nhieu/Datasets_1_loc_nhieu/database_sa2_Feb29_Mar06_Mar17_toApril06_cleaned_utt_0000187188-1.wav      85%  OK 
Extracting  /content/drive/MyDrive/Whisper/Datasets_1_loc_n

---Giải nén

In [ ]:
!unzip  /content/drive/MyDrive/Whisper/2000_Data_Goc.zip -d /content/drive/MyDrive/Whisper/2000_Data_Goc


Archive:  /content/drive/MyDrive/Whisper/2000_Data_Goc.zip
   creating: /content/drive/MyDrive/Whisper/2000_Data_Goc/Data_Goc/
  inflating: /content/drive/MyDrive/Whisper/2000_Data_Goc/Data_Goc/database_sa1_Jan08_Mar19_cleaned_utt_0000000005-1.txt  
  inflating: /content/drive/MyDrive/Whisper/2000_Data_Goc/Data_Goc/database_sa1_Jan08_Mar19_cleaned_utt_0000000005-1.wav  
  inflating: /content/drive/MyDrive/Whisper/2000_Data_Goc/Data_Goc/database_sa1_Jan08_Mar19_cleaned_utt_0000000028-1.txt  
  inflating: /content/drive/MyDrive/Whisper/2000_Data_Goc/Data_Goc/database_sa1_Jan08_Mar19_cleaned_utt_0000000028-1.wav  
  inflating: /content/drive/MyDrive/Whisper/2000_Data_Goc/Data_Goc/database_sa1_Jan08_Mar19_cleaned_utt_0000000033-1.txt  
  inflating: /content/drive/MyDrive/Whisper/2000_Data_Goc/Data_Goc/database_sa1_Jan08_Mar19_cleaned_utt_0000000033-1.wav  
  inflating: /content/drive/MyDrive/Whisper/2000_Data_Goc/Data_Goc/database_sa1_Jan08_Mar19_cleaned_utt_0000000045-1.txt  
  inflating: